In [67]:
import contextlib
import io
import json
import sys

from neo4j import GraphDatabase

In [68]:
@contextlib.contextmanager
def nostdout():
    save_stdout = sys.stdout
    sys.stdout = io.BytesIO()
    yield
    sys.stdout = save_stdout

In [69]:
with open("../hello_world/settings.conf") as fs:
    cfg = json.load(fs)

user, password = cfg["user"], cfg["pwd"]
# uri = f"bolt://{cfg['host']}:{cfg['port']}"
uri = f"bolt://localhost:{cfg['port']}"
message = cfg["hello_message"]

cfg


{'host': 'neo4j',
 'port': 7687,
 'user': 'neo4j',
 'pwd': 'admin',
 'hello_message': 'hello, world'}

In [140]:
# Entities
area = {'id': '1', 'parent_id': '113', 'name': 'Москва', 'areas': []}
area2 = {'id': '2', 'name': 'Санкт-Петербург'}
p_area = {'id': '113',
          'parent_id': None,
          'name': 'Россия'}
currency = {'name': 'RUR'}
employer = {'id': 4714154, 'name': 'Ельмикеев Евгений Иванович'}
schedule = {'id': 'remote', 'name': 'Удаленная работа'}
vacancy_type = {'id': 'open', 'name': 'Открытая'}
vacancy = {'id': '49154439',
           'name': 'Аналитик бизнес-процессов Amo CRM',
           'has_test': False,
           'salary_from': 70000,
           'salary_to': 120000,
           'published_at': '2021-11-04T14:52:20+0300',
           'created_at': '2021-11-04T14:52:20+0300',
           'requirement': 'Опыт работы от 1 года. Обязательное понимание ключевых метрик бизнеса. Опыт работы в amoCRM. Знание специфических метрик в отраслях будет...',
           'responsibility': 'Общаться с клиентами, понимать бизнес-задачу клиента. Формировать предложение по решению, составить Т3 для разработчиков. Настраивать сервисы; СRM-cистема...'
           }


vacancy2 = {'id': '48621910',
            'name': 'Финансовый аналитик',
            'has_test': False,
            'published_at': '2021-11-04T14:52:20+0300',
            'created_at': '2021-11-04T14:52:20+0300',
            'requirement': 'Высшее профессиональное образование (финансы, экономика, математика). Уверенный пользователь ПК (MSOffice), отличное владение пакетом Excel. Уверенный/продвинутый пользователь 1С, БИТ.Финанс...',
            'responsibility': 'Формирование управленческой отчетности, сбор и аналитика данных. Настраивание процесса бюджетирования, бюджетного контроля и управления. Построение управленческой отчетности для пользователей внутри...'
            }

In [71]:
driver = None
print("waiting for neo4j connection...")
while not driver:
    try:
        with nostdout():
            driver = GraphDatabase.driver(uri, auth=(user, password))
    except:
        pass

"neo4j connection established"

waiting for neo4j connection...


'neo4j connection established'

In [141]:
vac_q = f"CREATE (a:Vacancy) "
for i,j in vacancy2.items():
    if j is not None:
        vac_q += f"SET a.{i} = {repr(j)} "

vac_q += f"RETURN a "
q = vac_q
vac_q

"CREATE (a:Vacancy) SET a.id = '48621910' SET a.name = 'Финансовый аналитик' SET a.has_test = False SET a.published_at = '2021-11-04T14:52:20+0300' SET a.created_at = '2021-11-04T14:52:20+0300' SET a.requirement = 'Высшее профессиональное образование (финансы, экономика, математика). Уверенный пользователь ПК (MSOffice), отличное владение пакетом Excel. Уверенный/продвинутый пользователь 1С, БИТ.Финанс...' SET a.responsibility = 'Формирование управленческой отчетности, сбор и аналитика данных. Настраивание процесса бюджетирования, бюджетного контроля и управления. Построение управленческой отчетности для пользователей внутри...' RETURN a "

In [111]:
mrg_q ="""
MATCH (a:Vacancy {id:'48621910'}),
(b:Area {id:'1')
MERGE (a)-[r:SUBAREA_OF]->(b)
RETURN a

"""


q = mrg_q

In [142]:


transaction = lambda tx: tx.run(q).values()


with driver.session() as session:
    print(session.write_transaction(transaction))



[[<Node id=43 labels=frozenset({'Vacancy'}) properties={'has_test': False, 'responsibility': 'Формирование управленческой отчетности, сбор и аналитика данных. Настраивание процесса бюджетирования, бюджетного контроля и управления. Построение управленческой отчетности для пользователей внутри...', 'name': 'Финансовый аналитик', 'created_at': '2021-11-04T14:52:20+0300', 'id': '48621910', 'requirement': 'Высшее профессиональное образование (финансы, экономика, математика). Уверенный пользователь ПК (MSOffice), отличное владение пакетом Excel. Уверенный/продвинутый пользователь 1С, БИТ.Финанс...', 'published_at': '2021-11-04T14:52:20+0300'}>]]


In [66]:
driver.close()
"closed"

'closed'